# Dataset Preprocessing
In this notebook, the goal is to process the dataset so that the dataset is ready to be used for training. To achieve this goal, here are the steps that will be performed in this notebook:
1. Combine the CSV files of the dataset into one dataset
2. Preliminary analysis of the dataset
3. Dataset cleaning - clean missing values, duplicates, etc
4. Further processing, includes addressing the high class imbalance problem and rename the columns name (if necessary)
5. Save the processed dataset

note: The notebook will be seperated into two subsections, where the training dataset and the testing dataset will be processed separately. However, same steps will be applied to both dataset. 

In [1]:
# import the packages to be used
import os       # to create directories and remove files
import csv      # to remove extra columns in CSE-CIC-IDS2018 dataset
import numpy as np
import pandas as pd
import random

# set the random seed to ensure the result is reproducible
random.seed(10)
np.random.seed(10)

# CIC-IDS2017 Dataset

## Step 1. Combine the files of the dataset

In this step we will also address an issue where the files of the `CIC-IDS2017` dataset includes the replacement character (U+FFFD). This is because the dataset use the "–" character (Unicode code: U+2013), which cannot be processed by the Python-Pandas library. The replacement character will be replaced with the "-" character (Unicode code:45) when the files are being combined. The issue is addressed now instead of after importing the dataset as Dataframe because the dataset is very huge in size and the Pandas library is not optimized for such operation. 

In [2]:
# put the name of all files into a tuple
dataset = 'CIC-IDS2017'
csv_file_names = ('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX', 
                'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX',
                'Friday-WorkingHours-Morning.pcap_ISCX',
                'Monday-WorkingHours.pcap_ISCX',
                'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX',
                'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX',
                'Tuesday-WorkingHours.pcap_ISCX',
                'Wednesday-workingHours.pcap_ISCX')

In [3]:
# Define the function to combine seperate files of a dataset
# Besides combining the files, it has two additional functionality:
# 1. Replace the replacement character (\uFFFD) with '-', which is needed for the CIC-IDS2017 dataset
# 2. Down sample the dataset. If the reduce_sample_size parameter is set to true, 
#    only 10% of the dataset will be randomly selected and saved. 
def combine_csv_files(dataset: str, file_names: tuple, reduce_sample_size: bool = False):

    # create a new directory to place the combined dataset
    os.makedirs('./Dataset/dataset_combined', exist_ok=True)

    # remove the dataset if it already exist
    merged_dataset_directory = f'Dataset/dataset_combined/{dataset}.csv'
    if os.path.isfile(merged_dataset_directory):
        os.remove(merged_dataset_directory)
        print(f'original file({merged_dataset_directory}) has been removed')

    for (file_index, file_name) in enumerate(file_names):
        with open(f"Dataset/{dataset}/{file_name}.csv", 'r') as file, open(merged_dataset_directory, 'a') as out_file:
            for (line_index, line) in enumerate(file):
                
                # only the header of the first file will be taken
                if 'Label' in line or 'label' in line:
                    if file_index != 0 or line_index != 0:
                        continue
                elif reduce_sample_size:
                    if random.randint(1, 10) > 1:
                        continue
                # replace the replacement character (\uFFFD) with '-' if exist     
                out_file.write(line.replace(' ï¿½ ', '-'))
                

In [4]:
combine_csv_files(dataset=dataset, file_names=csv_file_names)

original file(Dataset/dataset_combined/CIC-IDS2017.csv) has been removed


## Step 2. Preliminary analysis

In [5]:
# put all csv files into a single dataframe
cic_ids2017 = pd.read_csv('Dataset/dataset_combined/CIC-IDS2017.csv')
cic_ids2017.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
print(f"Number of rows: {cic_ids2017.shape[0]}")
print(f"Number of columns: {cic_ids2017.shape[1]}")

Number of rows: 2830743
Number of columns: 79


In [7]:
print("Columns in the dataset:")
cic_ids2017.columns

Columns in the dataset:


Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

From the class distribution, we can clearly see that the CIC-IDS2017 dataset is very imbalance. The benign samples takes up a big portion of the dataset while some malicious samples like web attack contribute to a very small portion of the dataset. 

In [8]:
print('Class distribution:')
cic_ids2017[' Label'].value_counts()

Class distribution:


BENIGN                      2273097
DoS Hulk                     231073
PortScan                     158930
DDoS                         128027
DoS GoldenEye                 10293
FTP-Patator                    7938
SSH-Patator                    5897
DoS slowloris                  5796
DoS Slowhttptest               5499
Bot                            1966
Web Attack-Brute Force         1507
Web Attack-XSS                  652
Infiltration                     36
Web Attack-Sql Injection         21
Heartbleed                       11
Name:  Label, dtype: int64

In [9]:
print('Class distribution (normalized):')
cic_ids2017[' Label'].value_counts()/cic_ids2017.shape[0]*100

Class distribution (normalized):


BENIGN                      80.300366
DoS Hulk                     8.162981
PortScan                     5.614427
DDoS                         4.522735
DoS GoldenEye                0.363615
FTP-Patator                  0.280421
SSH-Patator                  0.208320
DoS slowloris                0.204752
DoS Slowhttptest             0.194260
Bot                          0.069452
Web Attack-Brute Force       0.053237
Web Attack-XSS               0.023033
Infiltration                 0.001272
Web Attack-Sql Injection     0.000742
Heartbleed                   0.000389
Name:  Label, dtype: float64

### Check for null value

In [10]:
cic_ids2017_null_count = cic_ids2017.isnull().sum()
cic_ids2017_null_count = cic_ids2017_null_count[cic_ids2017_null_count > 0]
print(f"Rows contain null value: \n{cic_ids2017_null_count}\n")

cic_ids2017_null_count = cic_ids2017_null_count / cic_ids2017.shape[0] * 100
print(f"Rows contain null value (percentage): \n{cic_ids2017_null_count}\n")

Rows contain null value: 
Flow Bytes/s    1358
dtype: int64

Rows contain null value (percentage): 
Flow Bytes/s    0.047973
dtype: float64



### Check for infinity value

In [11]:
print('Number of samples contains infinity value:')
np.isinf(cic_ids2017.iloc[:, :-2]).any(axis=1).sum()

Number of samples contains infinity value:


2867

### Check for columns that contain string values

Check for columns that is type `object`, which indicate the columns contain string value. The aim is to find if any column contain numeric and alphabetic value. Such column often include string value like '?' to indicate missing value, thus needed to be cleaned. 

In the CIC-IDS2017 dataset, only the Label column includes string value, hence no further cleaning will be needed. 

In [12]:
cic_ids2017.dtypes[(cic_ids2017.dtypes != 'int64') & (cic_ids2017.dtypes != 'float64')]

 Label    object
dtype: object

### Check for duplicates

Check for duplicated column

Although there are no two columns with the same name, there are actually two columns for `Fwd Header Length`. By manually inspecting all of the columns, we can find that there is one column called `Fwd Header Length` while there is another column called `Fwd Header Length.1`

In [13]:
# check for duplicated column
cic_ids2017.columns[cic_ids2017.columns.value_counts() > 1]

Index([], dtype='object')

In [14]:
# show all columns of the dataset to manually inspect it
cic_ids2017.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

Check for duplicate rows

In [15]:
cic_ids2017_duplicates = cic_ids2017[cic_ids2017.duplicated()]
cic_ids2017_duplicates.shape

(308381, 79)

From here we can see that the CIC-IDS2017 dataset contains quite a lot of duplicated entries. After looking into the number of duplicates according to the class of the samples, we have found that the duplicates are not specific to any class. Although 11% of duplicates is quite a lot, but these duplicates will be removed in the next step. The reason is that, 1. the dataset is large, even if we remove those samples, we still have sufficient samples to train our models. 2. duplicates will cause the models to bias towards them.  

In [16]:
print(f"{cic_ids2017_duplicates.shape[0]/cic_ids2017.shape[0]*100:.2f}% of rows are duplicates")

10.89% of rows are duplicates


In [17]:
cic_ids2017_duplicates[' Label'].value_counts()

BENIGN                    176613
PortScan                   68111
DoS Hulk                   58224
SSH-Patator                 2678
FTP-Patator                 2005
DoS slowloris                411
DoS Slowhttptest             271
Web Attack-Brute Force        37
Bot                           13
DDoS                          11
DoS GoldenEye                  7
Name:  Label, dtype: int64

## Step 3. Dataset cleaning

From the analysis in Step 2, it has been discovered that the dataset contains a small amount of entries contain infinity value or missing value. These entries will be removed as they only account for a small portion of the dataset. 

Besides that, it has been discovered that 11% of the dataset are duplicates. Despite the duplicates account for a big portion of the dataset, they are still removed from the dataset. 

In terms of the column of the dataset, there is one duplicated column, which will be dropped in this step. Besides that, some of the column names contain an extra space as the first character of the column name. The extra space will also be addressed here. 

In [18]:
# remove rows with missing value or infinity value
# by temporarily treating infinity value as null and use the dropna() function
with pd.option_context('mode.use_inf_as_na', True):
    cic_ids2017 = cic_ids2017.dropna(how='any')

cic_ids2017.shape

(2827876, 79)

In [19]:
cic_ids2017 = cic_ids2017.drop_duplicates()
cic_ids2017.shape

(2520798, 79)

In [20]:
# drop the duplicated column
cic_ids2017 = cic_ids2017.drop(' Fwd Header Length.1', axis=1)
cic_ids2017.shape

(2520798, 78)

In [21]:
# remove the extra space in the columns' name
cic_ids2017_columns = [column for column in cic_ids2017.columns]
for column_index, column in enumerate(cic_ids2017_columns):
    if column[0] == ' ':
        cic_ids2017_columns[column_index] = column[1:]

cic_ids2017.columns = cic_ids2017_columns
cic_ids2017.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

## Step 4. Dataset preparation

In this step, the main goal is to address the high class imbalance problem. To reduce the imbalance problem, major classes will be down sampled. Besides that, all attack samples will be combined. Hence, the dataset will only contain two label, 'benign' and 'malicious'. Besides that, 'benign' samples will be downsampled so that the ratio between the number of benign samples and malicious samples will be $1:1$

### Downsample the dataset

In [22]:

def downsample_dataset(dataset: pd.DataFrame, sample_count_per_class: pd.Series, max_sample: int) -> pd.DataFrame:
    dataset_downsampled = pd.DataFrame()
    for label, count in sample_count_per_class.items():

        # set the upper bound
        if count > max_sample:
            sample_size = max_sample
        else:
            sample_size = count

        sample = dataset[dataset['Label'] == label].sample(n=sample_size).reset_index(drop=True)
        dataset_downsampled = pd.concat([dataset_downsampled, sample])
    
    return dataset_downsampled

In [23]:
# get the number of sample for each class
sample_count_per_class = cic_ids2017['Label'].value_counts()
sample_count_per_class

BENIGN                      2095057
DoS Hulk                     172846
DDoS                         128014
PortScan                      90694
DoS GoldenEye                 10286
FTP-Patator                    5931
DoS slowloris                  5385
DoS Slowhttptest               5228
SSH-Patator                    3219
Bot                            1948
Web Attack-Brute Force         1470
Web Attack-XSS                  652
Infiltration                     36
Web Attack-Sql Injection         21
Heartbleed                       11
Name: Label, dtype: int64

In [24]:
cic_ids2017_attack = downsample_dataset(cic_ids2017, sample_count_per_class[1:], max_sample=100000)
cic_ids2017_benign = cic_ids2017[cic_ids2017['Label'] == 'BENIGN'].sample(n=cic_ids2017_attack.shape[0]).reset_index(drop=True)
cic_ids2017_downsampled = pd.concat([cic_ids2017_attack, cic_ids2017_benign])
del cic_ids2017_attack
del cic_ids2017_benign

print('Distribution of class after downsampling')
cic_ids2017_downsampled['Label'].value_counts()

Distribution of class after downsampling


BENIGN                      324881
DDoS                        100000
DoS Hulk                    100000
PortScan                     90694
DoS GoldenEye                10286
FTP-Patator                   5931
DoS slowloris                 5385
DoS Slowhttptest              5228
SSH-Patator                   3219
Bot                           1948
Web Attack-Brute Force        1470
Web Attack-XSS                 652
Infiltration                    36
Web Attack-Sql Injection        21
Heartbleed                      11
Name: Label, dtype: int64

In [25]:
print('Class distribution (normalized):')
cic_ids2017_downsampled['Label'].value_counts()/cic_ids2017_downsampled.shape[0]*100

Class distribution (normalized):


BENIGN                      50.000000
DDoS                        15.390251
DoS Hulk                    15.390251
PortScan                    13.958034
DoS GoldenEye                1.583041
FTP-Patator                  0.912796
DoS slowloris                0.828765
DoS Slowhttptest             0.804602
SSH-Patator                  0.495412
Bot                          0.299802
Web Attack-Brute Force       0.226237
Web Attack-XSS               0.100344
Infiltration                 0.005540
Web Attack-Sql Injection     0.003232
Heartbleed                   0.001693
Name: Label, dtype: float64

### Relabel the dataset

After relabeling the dataset, there will only be two classes, `malicious` and `benign`

In [26]:
cic_ids2017_downsampled.iloc[cic_ids2017_downsampled['Label'] != 'BENIGN', -1] = 'malicious'
cic_ids2017_downsampled.iloc[cic_ids2017_downsampled['Label'] == 'BENIGN', -1] = 'benign'

cic_ids2017_downsampled['Label'].value_counts()

benign       324881
malicious    324881
Name: Label, dtype: int64

## Step 5. Save the dataset

In [27]:
# function to save the cleaned dataset
def save_cleaned_dataset(dataframe: pd.DataFrame,dataset: str, tag: str = ""):
    # create a new directory to save the cleaned dataset
    os.makedirs('./Dataset/dataset_cleaned', exist_ok=True)

    if not(tag == ""):
        tag = "_" + tag

    dataframe.to_csv(f'Dataset/dataset_cleaned/{dataset}{tag}.csv', index=False)

In [28]:
save_cleaned_dataset(dataframe=cic_ids2017_downsampled, dataset='CIC-IDS2017')

# CSE-CIC-IDS2018 Dataset

The followings are the process to pre-process the CSE-CIC-IDS2018 dataset. The overall process is basically the same as that of the CIC-IDS2017 dataset. However, there are a few extra processing that is needed for the CSE-CIC-IDS2018 dataset. 

The most obvious extra processing is the need to align the columns of the CSE-CIC-IDS2018 dataset with that of the CIC-IDS2017 dataset. Hence, we will need to rename the column names of the 2018 dataset and ensure the sequence of the columns is exactly the same on both dataset. 

Moreover, the files of the 2018 dataset is more problematic. There are two main problems in the files:
1. In some of the files, there are duplicated header within one file. 
2. In the `Tuesday-20-02-2018_TrafficForML_CICFlowMeter.csv` file, there are 84 columns instead of 80 as of other files. 

The fist problem will be addressed when merging the files into one single CSV file by using the self-define `combine_csv_files()` function. The function will address the problem by ignoring all header except the first line of the first file. 

The second problem will be addressed in Step 0 by inspecting which 4 columns are extra and those columns will be removed. 

## Step 0. Clean the extra columns in the Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv file

### Load a normal file

Load a normal file so that we can know which columns are there in files with 80 columns. 

In [29]:
# since our interest is to get the columns in the file, so we only load a small number of rows
cse_cic_ids2018 = pd.read_csv(f'Dataset/CSE-CIC-IDS2018/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv', nrows=10)
cse_cic_ids2018_typical_columns = pd.Series(cse_cic_ids2018.columns, dtype='str')
print(cse_cic_ids2018.shape)

(10, 80)


### Load the problematic file

In [30]:
cse_cic_ids2018_20022018 = pd.read_csv('Dataset/CSE-CIC-IDS2018/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv', nrows=10)
cse_cic_ids2018_20022018_columns = pd.Series(cse_cic_ids2018_20022018.columns, dtype='str')

cse_cic_ids2018_20022018.shape

(10, 84)

### Check for columns that only exist in the problematic file. 

From the result below, we find that there are four extra columns that only exist in the problematic file. Besides that, the index of these columns shows that they are the first four columns in the file. As there are only four extra columns, it means that there are no missing columns in the problematic file. 

In [31]:
cse_cic_ids2018_20022018_columns[~cse_cic_ids2018_20022018_columns.isin(cse_cic_ids2018_typical_columns)]

0     Flow ID
1      Src IP
2    Src Port
3      Dst IP
dtype: object

### Address the issue

For performance reason, we use the CSV library to drop the first four columns of each row and save it to a new csv file. At the same time, let us fix the typo "Thuesday".

In [32]:
with open('Dataset/CSE-CIC-IDS2018/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv', 'r') as source, \
    open('Dataset/CSE-CIC-IDS2018/Tuesday-20-02-2018_TrafficForML_CICFlowMeter_dropped_first_four_columns.csv', 'w') as result:

    original_dataset = csv.reader(source)
    writer = csv.writer(result)

    for row in original_dataset:
        # exclude the first 4 columns when writing the file
        writer.writerow((row[4:]))

### Load the processed file to ensure the problem has been addressed

In [33]:
cse_cic_ids2018_20022018 = pd.read_csv('Dataset/CSE-CIC-IDS2018/Tuesday-20-02-2018_TrafficForML_CICFlowMeter_dropped_first_four_columns.csv', nrows=10)
print(cse_cic_ids2018_20022018.shape)
print(cse_cic_ids2018_20022018.columns)

(10, 80)
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg

## Step 1. Combine the files of the dataset

When combining the CSE-CIC-IDS2018 dataset, only 10% of the dataset will be used. This is because the 2018 dataset is too large to be handled by the hardware used in this work. Besides that, the CSE-CIC-IDS2018 dataset is only used for testing, 10% of the dataset is more than enough. 

In [34]:
dataset = 'CSE-CIC-IDS2018'
dataset_csv_files = ('Friday-02-03-2018_TrafficForML_CICFlowMeter',
                    'Friday-16-02-2018_TrafficForML_CICFlowMeter',
                    'Friday-23-02-2018_TrafficForML_CICFlowMeter',
                    'Tuesday-20-02-2018_TrafficForML_CICFlowMeter_dropped_first_four_columns',
                    'Thursday-01-03-2018_TrafficForML_CICFlowMeter',
                    'Thursday-15-02-2018_TrafficForML_CICFlowMeter',
                    'Thursday-22-02-2018_TrafficForML_CICFlowMeter',
                    'Wednesday-14-02-2018_TrafficForML_CICFlowMeter',
                    'Wednesday-21-02-2018_TrafficForML_CICFlowMeter',
                    'Wednesday-28-02-2018_TrafficForML_CICFlowMeter')

In [35]:
combine_csv_files(dataset, dataset_csv_files, reduce_sample_size=True)

original file(Dataset/dataset_combined/CSE-CIC-IDS2018.csv) has been removed


## Step 2. Preliminary analysis

In [36]:
# load the dataset into one DataFrame
cse_cic_ids2018 = pd.read_csv('Dataset/dataset_combined/CSE-CIC-IDS2018.csv')
cse_cic_ids2018.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49684,6,02/03/2018 08:47:38,281,2,1,38.0,0.0,38.0,0.0,...,20,0.0000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,Benign
1,443,6,02/03/2018 08:47:41,250,2,0,0.0,0.0,0.0,0.0,...,20,0.0000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,60860062,15,13,870.0,3306.0,535.0,0.0,...,20,130201.6667,148831.65440,434003.0,69408.0,10000000.0,16330.68406,10000000.0,9968389.0,Benign
3,443,6,02/03/2018 08:47:38,118281864,36,83,1022.0,108156.0,250.0,0.0,...,20,134521.0000,79961.04903,191062.0,77980.0,59000000.0,44452.97491,59000000.0,58900000.0,Benign
4,49745,6,02/03/2018 08:51:24,96328,2,1,38.0,0.0,38.0,0.0,...,20,0.0000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,Benign


In [37]:
print(f"Number of rows: {cse_cic_ids2018.shape[0]}")
print(f"Number of columns: {cse_cic_ids2018.shape[1]}")

Number of rows: 1622580
Number of columns: 80


In [38]:
print("Columns in the dataset:")
cic_ids2017.columns

Columns in the dataset:


Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

Just like the CIC-IDS2017 dataset, the CSE-CIC-IDS2018 dataset also suffer from the high class imbalance problem. 

In [39]:
print('Class distribution:')
cse_cic_ids2018['Label'].value_counts()

Class distribution:


Benign                      1347953
DDOS attack-HOIC              68801
DDoS attacks-LOIC-HTTP        57550
DoS attacks-Hulk              46014
Bot                           28539
FTP-BruteForce                19484
SSH-Bruteforce                18485
Infilteration                 16160
DoS attacks-SlowHTTPTest      14110
DoS attacks-GoldenEye          4154
DoS attacks-Slowloris          1076
DDOS attack-LOIC-UDP            163
Brute Force -Web                 59
Brute Force -XSS                 25
SQL Injection                     7
Name: Label, dtype: int64

In [40]:
print('Class distribution (normalized):')
cse_cic_ids2018['Label'].value_counts()/cse_cic_ids2018.shape[0]*100

Class distribution (normalized):


Benign                      83.074671
DDOS attack-HOIC             4.240222
DDoS attacks-LOIC-HTTP       3.546820
DoS attacks-Hulk             2.835854
Bot                          1.758866
FTP-BruteForce               1.200804
SSH-Bruteforce               1.139235
Infilteration                0.995945
DoS attacks-SlowHTTPTest     0.869603
DoS attacks-GoldenEye        0.256012
DoS attacks-Slowloris        0.066314
DDOS attack-LOIC-UDP         0.010046
Brute Force -Web             0.003636
Brute Force -XSS             0.001541
SQL Injection                0.000431
Name: Label, dtype: float64

### Check for null value

In [41]:
cse_cic_ids2018_null_count = cse_cic_ids2018.isnull().sum()
cse_cic_ids2018_null_count = cse_cic_ids2018_null_count[cse_cic_ids2018_null_count > 0]
print(f"Rows contain null value: \n{cse_cic_ids2018_null_count}\n")

cse_cic_ids2018_null_count = cse_cic_ids2018_null_count / cse_cic_ids2018.shape[0] * 100
print(f"Rows contain null value (percentage): \n{cse_cic_ids2018_null_count}\n")

Rows contain null value: 
Flow Byts/s    5975
dtype: int64

Rows contain null value (percentage): 
Flow Byts/s    0.368241
dtype: float64



### Check for infinity value

In [42]:
inf_count = np.isinf(cse_cic_ids2018.iloc[:, 3:-1]).any(axis=1).sum()
print(f'Number of rows includes infinity value: {inf_count}; {inf_count/cse_cic_ids2018.shape[0]*100:.2f}% of rows')

Number of rows includes infinity value: 9539; 0.59% of rows


### Check for columns that contain string values

The CSE-CIC-IDS2018 dataset contains two column of type `object`, which are the `Timestamp` and `Label`. Since both columns are not storing numerical value, it is very normal and no furter cleaning is needed. 

In [43]:
cse_cic_ids2018.dtypes[(cse_cic_ids2018.dtypes != 'int64') & (cse_cic_ids2018.dtypes != 'float64')]

Timestamp    object
Label        object
dtype: object

In [44]:
cse_cic_ids2018['Label'].unique()

array(['Benign', 'Bot', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk',
       'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection',
       'DDoS attacks-LOIC-HTTP', 'Infilteration', 'DoS attacks-GoldenEye',
       'DoS attacks-Slowloris', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DDOS attack-LOIC-UDP', 'DDOS attack-HOIC'], dtype=object)

In [45]:
cse_cic_ids2018['Timestamp']


0          02/03/2018 08:47:38
1          02/03/2018 08:47:41
2          02/03/2018 08:47:40
3          02/03/2018 08:47:38
4          02/03/2018 08:51:24
                  ...         
1622575    28/02/2018 01:48:05
1622576    28/02/2018 02:23:17
1622577    28/02/2018 04:05:10
1622578    28/02/2018 11:12:19
1622579    28/02/2018 01:48:05
Name: Timestamp, Length: 1622580, dtype: object

### Check for duplicates

Check for duplicated column

In [46]:
cse_cic_ids2018.columns[cse_cic_ids2018.columns.value_counts() > 1]

Index([], dtype='object')

Check for duplicate rows

In [47]:
cse_cic_ids2018_duplicates = cse_cic_ids2018[cse_cic_ids2018.duplicated()]
print(f"number of duplicated rows: {cse_cic_ids2018_duplicates.shape[0]}")
print(f"{cse_cic_ids2018_duplicates.shape[0]/cse_cic_ids2018.shape[0]*100:.2f}% of rows are duplicates")

number of duplicated rows: 17140
1.06% of rows are duplicates


## Step 3. Dataset cleaning

From the analysis in Step 2, it has been discovered that the CSE-CIC-IDS2018 dataset contains a small amount of rows with missing value or infinity value and a small amount of duplicates. These entries will be removed as they only account for a small portion of the dataset. 

In [48]:
# remove rows with null and infinity value
with pd.option_context('mode.use_inf_as_na', True):
    cse_cic_ids2018 = cse_cic_ids2018.dropna(how='any')

cse_cic_ids2018.shape

(1613041, 80)

In [49]:
cse_cic_ids2018 = cse_cic_ids2018.drop_duplicates()
cse_cic_ids2018.shape

(1595913, 80)

## Step 4. Dataset preparation

For the CSE-CIC-IDS2018 dataset, there are two goals in this step:
1. Just like the CIC-IDS2017 dataset, the 2018 dataset will be downsampled and all attack samples will be labeled as 'malicious'
2. The column names of the 2018 dataset will be modified to match that of the 2017 dataset. 

### Downsample the dataset

In [50]:
# get the number of sample for each class
sample_count_per_class = cse_cic_ids2018['Label'].value_counts()
sample_count_per_class

Benign                      1338122
DDOS attack-HOIC              68628
DDoS attacks-LOIC-HTTP        57550
DoS attacks-Hulk              45691
Bot                           28501
SSH-Bruteforce                16312
Infilteration                 16034
FTP-BruteForce                12368
DoS attacks-SlowHTTPTest       7251
DoS attacks-GoldenEye          4153
DoS attacks-Slowloris          1049
DDOS attack-LOIC-UDP            163
Brute Force -Web                 59
Brute Force -XSS                 25
SQL Injection                     7
Name: Label, dtype: int64

In [51]:
ids2018_attack = downsample_dataset(cse_cic_ids2018, sample_count_per_class[1:], max_sample=100000)
num_attack_sample = ids2018_attack.shape[0]
ids2018_benign = cse_cic_ids2018[cse_cic_ids2018['Label'] == 'Benign'].sample(n=num_attack_sample).reset_index(drop=True)
ids2018_downsampled = pd.concat([ids2018_attack, ids2018_benign])
del ids2018_attack
del ids2018_benign

print('Distribution of class after downsampling')
ids2018_downsampled['Label'].value_counts()

Distribution of class after downsampling


Benign                      257791
DDOS attack-HOIC             68628
DDoS attacks-LOIC-HTTP       57550
DoS attacks-Hulk             45691
Bot                          28501
SSH-Bruteforce               16312
Infilteration                16034
FTP-BruteForce               12368
DoS attacks-SlowHTTPTest      7251
DoS attacks-GoldenEye         4153
DoS attacks-Slowloris         1049
DDOS attack-LOIC-UDP           163
Brute Force -Web                59
Brute Force -XSS                25
SQL Injection                    7
Name: Label, dtype: int64

In [52]:
print('Class distribution (normalized):')
ids2018_downsampled['Label'].value_counts()/ids2018_downsampled.shape[0]*100

Class distribution (normalized):


Benign                      50.000000
DDOS attack-HOIC            13.310783
DDoS attacks-LOIC-HTTP      11.162143
DoS attacks-Hulk             8.862024
Bot                          5.527928
SSH-Bruteforce               3.163803
Infilteration                3.109884
FTP-BruteForce               2.398842
DoS attacks-SlowHTTPTest     1.406372
DoS attacks-GoldenEye        0.805497
DoS attacks-Slowloris        0.203459
DDOS attack-LOIC-UDP         0.031615
Brute Force -Web             0.011443
Brute Force -XSS             0.004849
SQL Injection                0.001358
Name: Label, dtype: float64

### Relabel the dataset

In [53]:
# replace the label of all attack class to 'malicious'
ids2018_downsampled.iloc[ids2018_downsampled['Label'] != 'Benign', -1] = 'malicious'
ids2018_downsampled.iloc[ids2018_downsampled['Label'] == 'Benign', -1] = 'benign'
ids2018_downsampled['Label'].value_counts()

benign       257791
malicious    257791
Name: Label, dtype: int64

### Align the columns name with CIC-IDS2017 dataset

In [54]:
ids2018_columns = pd.Series(ids2018_downsampled.columns, dtype='str')
ids2017_columns = pd.Series(cic_ids2017.columns, dtype='str')

In [55]:
column_mapping = {
    "Dst": "Destination",
    "TotLen": "Total Length",
    "Tot": "Total", 
    "Pkt": "Packet",
    "Len": "Length",
    "Cnt": "Count", 
    "Var": "Variance",
    "Total Bwd Packets": "Total Backward Packets", 
    "Totalal Lengthgth Fwd Packets": "Total Length of Fwd Packets", 
    "Totalal Lengthgth Bwd Packets": "Total Length of Bwd Packets", 
    "Flow Byts/s": "Flow Bytes/s",
    "Packet Size Avg": "Average Packet Size",
    "Fwd Seg Size Avg": "Avg Fwd Segment Size",
    "Bwd Seg Size Avg": "Avg Bwd Segment Size",
    "Fwd Byts/b Avg": "Fwd Avg Bytes/Bulk",
    "Fwd Packets/b Avg": "Fwd Avg Packets/Bulk",
    "Fwd Blk Rate Avg": "Fwd Avg Bulk Rate", 
    "Bwd Byts/b Avg": "Bwd Avg Bytes/Bulk",
    "Bwd Packets/b Avg": "Bwd Avg Packets/Bulk",
    "Bwd Blk Rate Avg": "Bwd Avg Bulk Rate", 
    "Init Fwd Win Byts": "Init_Win_bytes_forward", 
    "Init Bwd Win Byts": "Init_Win_bytes_backward",
    "Fwd Act Data Packets": "act_data_pkt_fwd",
    "Fwd Seg Size Min": "min_seg_size_forward",
    "Subflow Fwd Byts": "Subflow Fwd Bytes", 
    "Subflow Bwd Byts": "Subflow Bwd Bytes"
}

In [56]:
# rename the columns of the 2018 dataset
for original_value in column_mapping:
    ids2018_columns = ids2018_columns.replace({original_value: column_mapping[original_value]}, regex=True)

ids2018_columns[40] = 'Min Packet Length'
ids2018_columns[41] = 'Max Packet Length'

Check for extra columns in the CSE-CIC-IDS2018 dataset

In [57]:
ids2018_columns[~ids2018_columns.isin(ids2017_columns)]

1     Protocol
2    Timestamp
dtype: object

Check if the CSE-CIC-IDS2018 dataset miss any column that is in the CIC-IDS2017 dataset

In [58]:
ids2017_columns[~ids2017_columns.isin(ids2018_columns)]

Series([], dtype: object)

Change the column name of the dataframe and drop the additional columns

In [59]:
ids2018_downsampled.columns = ids2018_columns
# drop the additional columns
ids2018_downsampled = ids2018_downsampled.drop(['Protocol', 'Timestamp'], axis=1).copy()
ids2018_downsampled.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,1794,3,4,309.0,935.0,309.0,0.0,103.000000,178.401233,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
1,80,1205,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
2,80,18083,3,4,298.0,935.0,298.0,0.0,99.333333,172.050380,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
3,80,23013,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
4,80,17255,3,4,326.0,935.0,326.0,0.0,108.666667,188.216188,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious


### Ensure the order of the columns are exactly the same for both dataset

In [60]:
ids2018_downsampled = ids2018_downsampled.reindex(columns=ids2017_columns)

## Step 5. Save the dataset

In [61]:
save_cleaned_dataset(ids2018_downsampled, 'CSE-CIC-IDS2018')